In [1]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor

import warnings

In [2]:
df = pd.read_csv(r'C:\Users\Mahesh\Desktop\student_maths\artifacts\raw.csv')

In [3]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
X = df.drop(['math_score'], axis=1)
y = df['math_score']

In [5]:
print("Categories in 'gender' variable:     ",end=" " )
print(df['gender'].unique())

print("Categories in 'race_ethnicity' variable:  ",end=" ")
print(df['race_ethnicity'].unique())

print("Categories in'parental level of education' variable:",end=" " )
print(df['parental_level_of_education'].unique())

print("Categories in 'lunch' variable:     ",end=" " )
print(df['lunch'].unique())

print("Categories in 'test preparation course' variable:     ",end=" " )
print(df['test_preparation_course'].unique())

Categories in 'gender' variable:      ['female' 'male']
Categories in 'race_ethnicity' variable:   ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in'parental level of education' variable: ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in 'lunch' variable:      ['standard' 'free/reduced']
Categories in 'test preparation course' variable:      ['none' 'completed']


In [6]:
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns
print("Numerical features: ", num_features) 
print("Categorical features: ", cat_features)

Numerical features:  Index(['reading_score', 'writing_score'], dtype='object')
Categorical features:  Index(['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course'],
      dtype='object')


In [7]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ])

In [8]:
X_transformed = preprocessor.fit_transform(X)
X_transformed

array([[ 0.19399858,  0.39149181,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       [ 1.42747598,  1.31326868,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 1.77010859,  1.64247471,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       ...,
       [ 0.12547206, -0.20107904,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.60515772,  0.58901542,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 1.15336989,  1.18158627,  0.        , ...,  0.        ,
         0.        ,  1.        ]])

In [9]:
X_transformed.shape

(1000, 14)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

In [11]:
X_train.shape

(800, 14)

In [12]:
X_test.shape

(200, 14)

In [13]:
def evaluate_model(true, predicted):
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2 = r2_score(true, predicted)
    mae = mean_absolute_error(true, predicted)
    
    print(f"RMSE: {rmse:.2f}")
    print(f"R^2: {r2:.2f}")
    print(f"MAE: {mae:.2f}")
    
    return rmse, r2, mae

In [ ]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    #"XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2)) 
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

RMSE: 5.32
R^2: 0.87
MAE: 4.27
RMSE: 5.39
R^2: 0.88
MAE: 4.21
Model performance for Training set
- Root Mean Squared Error: 0.8743
- Mean Absolute Error: 5.3231
- R2 Score: 4.2667
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.8804
- Mean Absolute Error: 5.3940
- R2 Score: 4.2148


RMSE: 6.59
R^2: 0.81
MAE: 5.21
RMSE: 6.52
R^2: 0.83
MAE: 5.16
Model performance for Training set
- Root Mean Squared Error: 0.8071
- Mean Absolute Error: 6.5938
- R2 Score: 5.2063
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.8253
- Mean Absolute Error: 6.5197
- R2 Score: 5.1579


RMSE: 5.32
R^2: 0.87
MAE: 4.26
RMSE: 5.39
R^2: 0.88
MAE: 4.21
Model performance for Training set
- Root Mean Squared Error: 0.8743
- Mean Absolute Error: 5.3235
- R2 Score: 4.2650
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.8805
- Mean Absolute Error: 5.3936
- R2 Score: 4.2125




RMSE: 5.57
R^2: 0.86
MAE: 4.45
RMSE: 7.38
R^2: 0.78
MAE: 5.71
Model performance for Training set
- Root Mean Squared Error: 0.8626
- Mean Absolute Error: 5.5660
- R2 Score: 4.4493
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.7763
- Mean Absolute Error: 7.3774
- R2 Score: 5.7110


RMSE: 0.28
R^2: 1.00
MAE: 0.02
RMSE: 7.63
R^2: 0.76
MAE: 6.14
Model performance for Training set
- Root Mean Squared Error: 0.9997
- Mean Absolute Error: 0.2795
- R2 Score: 0.0187
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.7605
- Mean Absolute Error: 7.6341
- R2 Score: 6.1400


RMSE: 2.31
R^2: 0.98
MAE: 1.84
RMSE: 6.04
R^2: 0.85
MAE: 4.73
Model performance for Training set
- Root Mean Squared Error: 0.9762
- Mean Absolute Error: 2.3144
- R2 Score: 1.8429
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.8500
- Mean Absolute Error: 6.0424
- R2 Score: 4.7319


RMSE: 3